In [ ]:
# http://zhaoxuhui.top/blog/2018/09/11/PyTorchNote2.html

import torch
from matplotlib import pyplot as plt
import torch.nn.functional as F
import pandas as pd

#pd.options.mode.chained_assignment = 'raise'
pd.options.mode.chained_assignment = None

In [ ]:
# define net
TRAIN_TIMES = 3000
INPUT_FEATURE_DIM = 75
OUTPUT_FEATURE_DIM = 2
NEURON_NUM = 256
LEARNING_RATE = 0.1



net = torch.nn.Sequential(
    torch.nn.Linear(INPUT_FEATURE_DIM, NEURON_NUM),
    torch.nn.ReLU(),
    torch.nn.Linear(NEURON_NUM, OUTPUT_FEATURE_DIM)
)




In [ ]:
def make_clear_df(file, symbol):
    all_df = pd.read_parquet(file)
    aim_df = all_df[all_df['symbol'] == symbol]
    aim_df.loc[:, 'v'] = aim_df.loc[:, 'volume'].diff()
    columns = []
    f_columns = []
    feature_columns = []
    for i in range(5):
        aim_df.loc[:, 'av'+str(i)] = aim_df.loc[:, 'askVolume1'].shift(i)
        aim_df.loc[:, 'bv'+str(i)] = aim_df.loc[:, 'bidVolume1'].shift(i)
        aim_df.loc[:, 'v'+str(i)] = aim_df.loc[:, 'v'].shift(i)
        columns.append('av' + str(i))
        columns.append("bv" + str(i))
        columns.append("v" + str(i))
        feature_columns.append("av" + str(i))
        feature_columns.append("bv" + str(i))
        feature_columns.append("v" + str(i))
    for i in range(60):
        aim_df.loc[:, 'p'+str(i)] = aim_df.loc[:, 'lastPrice'].shift(i) - aim_df.loc[:, 'lastPrice']
        columns.append("p" + str(i))
        feature_columns.append("p" + str(i))
    for i in range(30):
        aim_df.loc[:, 'f'+str(i)] = aim_df.loc[:, 'lastPrice'].shift(-i) - aim_df.loc[:, 'lastPrice']
        columns.append("f" + str(i))
        f_columns.append("f" + str(i))
    hist = aim_df.loc[:, columns].dropna()
    hist.loc[:, 'min'] = hist.loc[:, f_columns].apply(min, axis=1)
    hist.loc[:, 'max'] = hist.loc[:, f_columns].apply(max, axis=1)
    return hist, feature_columns



In [ ]:
# prepare data
t, f_cs = make_clear_df("../data/2023-01-12-12.tick", 'CF305')


In [ ]:
t.head()

In [ ]:
len(f_cs)

In [ ]:
x_data = torch.tensor(t[f_cs].values).to(torch.float32)
y_data = torch.tensor(t[['min', 'max']].values).to(torch.float32)

In [50]:
# 训练网络
# 这里也可以使用其它的优化方法
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
# 定义一个误差计算方法
loss_func = torch.nn.MSELoss()

for i in range(TRAIN_TIMES):
    # 输入数据进行预测
    prediction = net(x_data)
    # 计算预测值与真值误差，注意参数顺序问题
    # 第一个参数为预测值，第二个为真值
    loss = loss_func(prediction, y_data)

    if i % 200:
        print(loss)
    # 开始优化步骤
    # 每次开始优化前将梯度置为0
    optimizer.zero_grad()
    # 误差反向传播
    loss.backward()
    # 按照最小loss优化参数
    optimizer.step()

tensor(10.6608, grad_fn=<MseLossBackward0>)
tensor(10.6515, grad_fn=<MseLossBackward0>)
tensor(10.6529, grad_fn=<MseLossBackward0>)
tensor(10.6566, grad_fn=<MseLossBackward0>)
tensor(10.6552, grad_fn=<MseLossBackward0>)
tensor(10.6521, grad_fn=<MseLossBackward0>)
tensor(10.6508, grad_fn=<MseLossBackward0>)
tensor(10.6520, grad_fn=<MseLossBackward0>)
tensor(10.6535, grad_fn=<MseLossBackward0>)
tensor(10.6535, grad_fn=<MseLossBackward0>)
tensor(10.6522, grad_fn=<MseLossBackward0>)
tensor(10.6510, grad_fn=<MseLossBackward0>)
tensor(10.6509, grad_fn=<MseLossBackward0>)
tensor(10.6517, grad_fn=<MseLossBackward0>)
tensor(10.6523, grad_fn=<MseLossBackward0>)
tensor(10.6522, grad_fn=<MseLossBackward0>)
tensor(10.6515, grad_fn=<MseLossBackward0>)
tensor(10.6509, grad_fn=<MseLossBackward0>)
tensor(10.6509, grad_fn=<MseLossBackward0>)
tensor(10.6513, grad_fn=<MseLossBackward0>)
tensor(10.6516, grad_fn=<MseLossBackward0>)
tensor(10.6515, grad_fn=<MseLossBackward0>)
tensor(10.6511, grad_fn=<MseLoss

In [ ]:
def evaluator(net, file_path, symbol):
    t, f_cs = make_clear_df(file_path, symbol)
    x_data = torch.tensor(t[f_cs].values).to(torch.float32)
    y_data = torch.tensor(t[['min', 'max']].values).to(torch.float32)
    prediction = net(x_data)
    loss = loss_func(prediction, y_data)
    return loss

In [51]:
import os
for f in os.listdir("../data/"):
    if f.endswith(".tick"):
        print(f, ":", evaluator(net, "../data/" + f, 'rb2305'))

2023-01-03-08.tick : tensor(108.8801, grad_fn=<MseLossBackward0>)
2023-01-03-12.tick : tensor(6.2858, grad_fn=<MseLossBackward0>)
2023-01-04-12.tick : tensor(6.2682, grad_fn=<MseLossBackward0>)
2023-01-04-20.tick : tensor(6.2802, grad_fn=<MseLossBackward0>)
2023-01-05-08.tick : tensor(67.0651, grad_fn=<MseLossBackward0>)
2023-01-05-12.tick : tensor(14.7708, grad_fn=<MseLossBackward0>)
2023-01-05-20.tick : tensor(6.1819, grad_fn=<MseLossBackward0>)
2023-01-06-08.tick : tensor(363.3564, grad_fn=<MseLossBackward0>)
2023-01-06-12.tick : tensor(6.2707, grad_fn=<MseLossBackward0>)
2023-01-06-20.tick : tensor(6.8507, grad_fn=<MseLossBackward0>)
2023-01-09-08.tick : tensor(47.9341, grad_fn=<MseLossBackward0>)
2023-01-09-12.tick : tensor(9.0141, grad_fn=<MseLossBackward0>)
2023-01-09-20.tick : tensor(39.8875, grad_fn=<MseLossBackward0>)
2023-01-10-08.tick : tensor(12.8999, grad_fn=<MseLossBackward0>)
2023-01-10-12.tick : tensor(8.7595, grad_fn=<MseLossBackward0>)
2023-01-10-20.tick : tensor(6.5

In [53]:
t1 = torch.tensor(range(10)).to(torch.float32)
t2 = torch.tensor(range(1, 11)).to(torch.float32)

lo = loss_func(t1, t2)
print(lo)

tensor(1.)
